In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd /content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN

/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN


In [0]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import numpy as np
import librosa as lb
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.optimizers import Adam,SGD
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Lambda
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers.merge import add, concatenate
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.utils import to_categorical,Sequence
import pandas as pd
import math
import sincnet
# from tensorflow import set_random_seed
from keras import models, layers
import numpy as np
import sincnet
from keras.layers import Dense, Dropout, Activation
from keras.layers import MaxPooling1D,MaxPooling2D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten
from keras.layers import InputLayer, Input
from keras.models import Model
from tensorflow.python.keras.utils.data_utils import Sequence
from sklearn.preprocessing import StandardScaler
import pywt
from sklearn.decomposition import PCA
import sys

In [0]:
def framing_windowing(signal):
    pre_emphasis = 0.97
    frame_size = 256
    frame_stride = 128
    nfilt = 20
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    frame_length, frame_step = frame_size, frame_stride  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
#     print(num_frames)
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    frames *= np.hamming(frame_length)
    # print(frames.shape)
    return frames

In [0]:
def tkeo(a):

    """
    Calculates the TKEO of a given recording by using 2 samples.
    See Li et al., 2007
    Arguments:
    a 			--- 1D numpy array.
    Returns:
    1D numpy array containing the tkeo per sample
    """
    # Create two temporary arrays of equal length, shifted 1 sample to the right
    # and left and squared:
    i = a[1:-1]*a[1:-1]
    j = a[2:]*a[:-2]
    # Calculate the difference between the two temporary arrays:
    aTkeo = i-j
    return aTkeo

In [0]:
def Wavelet_1d(signal):
  import numpy
  pre_emphasis = 0.97
  frame_size = 256
  frame_stride = 128
  nfilt = 20
  NFFT = 511
  sample_rate = 16000
  signal = framing_windowing(signal)
  # signal = tkeo(signal)
  mel = lb.filters.mel(sr=sample_rate, n_fft=NFFT, n_mels=nfilt)
  audio_features1 = []
  audio_features2 = []
  audio_features3 = []
  audio_features4 = []
  # print("signal shape",signal.shape)
  for f in signal:
    tke = tkeo(f)
    data_std = StandardScaler().fit_transform(tke.reshape(-1,1)).reshape(1,-1)[0]            
    wptree = pywt.WaveletPacket(data=data_std, wavelet='db1', mode='symmetric')
    level1 = wptree.get_level(1, order = "freq")
    level2 = wptree.get_level(2, order = "freq")
    level3 = wptree.get_level(3, order = "freq")
    level4 = wptree.get_level(4, order = "freq")
    # print("level1 data array:",np.array(level1).shape)
    # print("level2 data array:",np.array(level2).shape)
    # print("level3 data array:",np.array(level3).shape)
    # print("level4 data array:",np.array(level4).shape)          
      #Feature extraction for each node
    frame_features1 = []
    frame_features2 = []
    frame_features3 = []
    frame_features4 = []        
    for node in level1:
      data_wp = node.data
      # print("WP data:",np.array(data_wp).shape)
      # Features group
      frame_features1.extend(data_wp)
    # print("frame_features1",np.array(frame_features1).shape)
    mag_frames = numpy.absolute(frame_features1)  # Magnitude of the FFT
    pow_frames = numpy.abs((mag_frames) ** 2)
    z = mel.shape[1] - pow_frames.shape[0]
    pow_frames = np.pad(pow_frames,[(0,z)],'constant', constant_values=0)
    # print("pow_frames",pow_frames.shape)
    mel_scaled_features = mel.dot(pow_frames)
    audio_features1.append(mel_scaled_features)
    

    for node in level2:
      data_wp = node.data
      # print("WP data:",np.array(data_wp).shape)
      # Features group
      frame_features2.extend(data_wp)
    mag_frames = numpy.absolute(frame_features2)  # Magnitude of the FFT
    pow_frames = numpy.abs((mag_frames) ** 2)
    mel_scaled_features = mel.dot(pow_frames)
    audio_features2.append(mel_scaled_features)
    for node in level3:
      data_wp = node.data
      # print("WP data:",np.array(data_wp).shape)
      # Features group
      frame_features3.extend(data_wp)
    mag_frames = numpy.absolute(frame_features3)  # Magnitude of the FFT
    pow_frames = numpy.abs((mag_frames) ** 2)
    mel_scaled_features = mel.dot(pow_frames)
    audio_features3.append(mel_scaled_features)
    for node in level4:
      data_wp = node.data
      # print("WP data:",np.array(data_wp).shape)
      # Features group
      frame_features4.extend(data_wp)
    mag_frames = numpy.absolute(frame_features4)  # Magnitude of the FFT
    pow_frames = numpy.abs((mag_frames) ** 2)
    mel_scaled_features = mel.dot(pow_frames)
    audio_features4.append(mel_scaled_features)
    
  # print("audio_features1:",np.array(audio_features1).shape)
  # print("audio_features1:",np.array(audio_features2).shape)
  # print("audio_features1:",np.array(audio_features3).shape)
  # print("audio_features1:",np.array(audio_features4).shape)
#     print("hello")
  log_energy1 = numpy.log10(audio_features1)
  log_energy1 = pd.DataFrame(log_energy1)
  log_energy2 = numpy.log10(audio_features2)
  log_energy2 = pd.DataFrame(log_energy2)
  log_energy3 = numpy.log10(audio_features3)
  log_energy3 = pd.DataFrame(log_energy3)
  log_energy4 = numpy.log10(audio_features4)
  log_energy4 = pd.DataFrame(log_energy4)
  pd.set_option('use_inf_as_na', True)
  log_energy1=log_energy1.fillna(log_energy1.mean())
  log_energy1 = np.array(log_energy1)
  log_energy2=log_energy2.fillna(log_energy2.mean())
  log_energy2 = np.array(log_energy2)
  log_energy3=log_energy3.fillna(log_energy3.mean())
  log_energy3 = np.array(log_energy3)
  log_energy4=log_energy4.fillna(log_energy4.mean())
  log_energy4 = np.array(log_energy4)
  # print("log_energy1: ",log_energy1.shape)
  # print("log_energy2: ",log_energy2.shape)
  # print("log_energy3: ",log_energy3.shape)
  # print("log_energy4: ",log_energy4.shape)
  # signals_level1[count] = np.array(log_energy1)
  # signals_level2[count] = np.array(log_energy2)
  # signals_level3[count] = np.array(log_energy3)
  # signals_level4[count] = np.array(log_energy4)
  # print(signals_level1.shape)
  # print(signals_level2.shape)
  # print(signals_level3.shape)
  # print(signals_level4.shape)  

  return log_energy1,log_energy2,log_energy3,log_energy4

def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 590, 20]), tuple([None, 590, 20]), 
            tuple([None, 590, 20]), tuple([None, 590, 20])]

In [0]:
train_labels = np.load("/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN/temp_npy/train_label.npy")
trimmed_audio = np.load("/content/drive/My Drive/SA/Code/spoof_detection_deep_features/WaveletCNN/temp_npy/trimmed_audio.npy",mmap_mode='r')

In [62]:
wpt = Wavelet_1d(trimmed_audio[1])
print("wpt shape",wpt.shape)



: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

wpt shape (4, 590, 20)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [77]:
wpt_levels_data = []
for count,i in enumerate(trimmed_audio[:100]):
  if count%10 == 0:
    print(count)
  level1,level2,level3,level4 = Wavelet_1d(i)
  wpt_levels_data.append([level1,level2,level3,level4])
wpt_levels_data = np.array(wpt_levels_data)
print(wpt_levels_data.shape)

0
10
20
30
40
50
60
70
80
90
(100, 4, 590, 20)
